In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage import feature
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
train_csv_path = '/home/vandita/NandaniSharma/HOGTrain.csv'

# Load the CSV file into a Pandas DataFrame
train_df = pd.read_csv(train_csv_path)

# Load validation data
val_df = pd.read_csv('/home/vandita/NandaniSharma/HOGVal.csv')
X_val = val_df.drop('label', axis=1)
y_val = val_df['label']

# Standardize features
scaler = StandardScaler()
X_val_std = scaler.fit_transform(X_val)

# Define the parameter grid for GridSearchCV
param_grid = {
    'estimator__C': [1, 10, 100],  # You can specify other values as well
    'estimator__kernel': ['linear', 'rbf']  # Choose appropriate kernels
}

# Perform GridSearchCV to find the best parameters
svm = SVC(probability=True)
ovr_svm = OneVsRestClassifier(svm)
grid_search = GridSearchCV(ovr_svm, param_grid, cv=5)
grid_search.fit(X_val_std, y_val)

# Get the best parameters
best_C = grid_search.best_params_['estimator__C']
best_kernel = grid_search.best_params_['estimator__kernel']

# Train One-vs-Rest SVM classifier with best parameters using the combined train and validation data
X_train_combined = np.concatenate((train_df.drop('label', axis=1), X_val))
y_train_combined = np.concatenate((train_df['label'], y_val))
X_train_combined_std = scaler.transform(X_train_combined)

svm_classifier = OneVsRestClassifier(SVC(C=best_C, kernel=best_kernel, probability=True))
svm_classifier.fit(X_train_combined_std, y_train_combined)

# Load test data
test_df = pd.read_csv('/home/vandita/NandaniSharma/HOGTest.csv')
X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Standardize test data
X_test_std = scaler.transform(X_test)

# Predict and evaluate on test data
y_pred = svm_classifier.predict(X_test_std)
accuracy_svm = accuracy_score(y_test, y_pred)
print(f'Accuracy of One-vs-Rest SVM classifier on Test.csv: {accuracy_svm:.2f}')
